In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import euclidean_distances, r2_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("train_dataset_train.csv")

In [3]:
#сначала разберемся с датами
df.pass_dttm = pd.to_datetime(df.pass_dttm)

def hour_period(hour):
    if hour < 10:
        return 0.25
    if hour < 17:
        return 0.5
    if hour < 21:
        return 0.75
    return 1

#построим фичи из даты
df['weekday'] = df['pass_dttm'].apply(lambda x : 0 if x.weekday() < 5 else 1) 
df['pass_time'] = df['pass_dttm'].apply(lambda x : hour_period(x.hour)) 
df.drop(columns=['pass_dttm'], inplace=True)

In [4]:
#очевидный мусор
df.drop(columns=['ticket_id','entrance_id', 'entrance_nm', 'id', 'line_nm', 'station_nm'], inplace=True)

In [5]:
df.head(5)

,ticket_type_nm,station_id,line_id,time_to_under,label,weekday,pass_time
0,Пропуск FacePay,11007,11,216.316667,8001,0,0.25
1,СК учащегося 30 дней,2006,2,648.183333,9011,0,0.25
2,БСК дружинника г.Москвы,2006,2,865.333333,7022,0,0.25
3,30 дней,2006,2,1048.233333,2022,0,0.25
4,КОШЕЛЕК,2006,2,965.600000,2017,0,0.25


In [6]:
df.head()

,ticket_type_nm,station_id,line_id,time_to_under,label,weekday,pass_time
0,Пропуск FacePay,11007,11,216.316667,8001,0,0.25
1,СК учащегося 30 дней,2006,2,648.183333,9011,0,0.25
2,БСК дружинника г.Москвы,2006,2,865.333333,7022,0,0.25
3,30 дней,2006,2,1048.233333,2022,0,0.25
4,КОШЕЛЕК,2006,2,965.600000,2017,0,0.25


In [7]:
ticket_types = df['ticket_type_nm'].unique()
# print(len(ticket_types))
# pd.get_dummies(df, columns=['ticket_type_nm'])

In [8]:
#опять какой-то мусор подсунули, много дубликатов
dict = {
    'КОШЕЛЕК': 'КОШЕЛЕК',
    'ББК': 'КОШЕЛЕК',
    '30 дней СК студента': 'СК студента',
    '90 дней СК студента': 'СК студента',
    'СК студента 30 дней': 'СК студента',
    '90 дней СК аспиранта': 'СК студента',
    'СК аспиранта 90 дней': 'СК студента',
    'СК аспиранта 30 дней': 'СК студента',
    '30 дней СК аспиранта': 'СК студента',
    '30 дней СК ординатора': 'СК студента',
    '90 дней СК ординатора': 'СК студента',
    'СК студента 90 дней': 'СК студента',
    'СК ординатора 30 дней': 'СК студента',
    'СК ординатора 90 дней': 'СК студента',
    '30 дней СК ассистента-стажера': 'СК сотрудника',
    'СК ассистента-стажера 30 дней': 'СК сотрудника',
    'СК ассистента-стажера 90 дней': 'СК сотрудника',
    '30 дней СК учащегося': 'СК учащегося',
    'СК учащегося 90 дней': 'СК учащегося',
    '90 дней СК учащегося': 'СК учащегося',
    'СК учащегося 30 дней': 'СК учащегося',
    '30 дней ЕДИНЫЙ ТК': 'ТК',
    '30 дней': 'ТК',
    '90 дней ЕДИНЫЙ ТК': 'ТК',
    '365 дней ЕДИНЫЙ ТК': 'ТК',
    '365 дней': 'ТК',
    '90 дней': 'ТК',
    'Ультралайт Единый (70)': 'ТК',
    '30 дней  Пригород': 'ТК',
    '90 дней  Пригород': 'ТК',
    '60 поездок ЕДИНЫЙ ТК': 'ТК',
    '30 дней Пригород ТК': 'ТК',
    'ВЕСБ МОСКВА (7мин)': 'ТК',
    '1 сутки ЕДИНЫЙ ТК': 'ТК',
    '3 суток Единый ТК': 'ТК',
    '30 дней Пригород': 'ТК',
    '90 дней Пригород ТК': 'ТК',
    '1 сутки Пригород ТК': 'ТК',
    'Билет 1 сутки ЕДИНЫЙ': 'ТК',
    'Временный билет ММ': 'КОШЕЛЕК',
    '90 дней Пригород': 'ТК',
    '365 дней Пригород ТК': 'ТК',
    'Билет 3 суток ЕДИНЫЙ': 'ТК',
    'Социальная карта жителя Моск. области': 'СК',
    'Социальная карта москвича': 'СК',
    'ВЕСБ МО (тип 5)': 'ТК',
    '365 дней  Пригород': 'ТК',
    'ВЕСБ МО (7 мин)': 'ТК',
    'Социальная карта жителя Моск. области с сопровождающим': 'СК',
    'Социальная карта москвича с сопровождающим': 'СК',
    '365 дней Пригород': 'ТК',
    'ВЕСБ МО с сопровождающим': 'ТК',
    'ВЕСБ МОСКВА': 'ТК',
    'Безналичная транспортная карта': 'КОШЕЛЕК',
    '365 дней Пригород': 'ТК',
    'Пропуск сотрудника  метрополитена': 'СК сотрудника',
    'Пропуск сотрудника метрополитена': 'СК сотрудника',
    'Пропуск  сотрудника МЦК': 'СК сотрудника',
    'Пропуск сотрудника МЦК': 'СК сотрудника',
    'ВЛБ МОСКВА': 'ТК',
    'Пропуск руководителя метрополитена': 'СК сотрудника',
    'СК для сотрудника ГУВД г.Москвы': 'СК сотрудника',
    'БСК дружинника г.Москвы': 'СК сотрудника',
    'Пропуск сотрудника УВД по охране Метрополитена': 'СК сотрудника',
    'Пропуск FacePay': 'КОШЕЛЕК',
    'ВЕСБ МО (тип 6)': 'ТК'
}
df['ticket_type_nm'] = df['ticket_type_nm'].apply(lambda x: dict[x])

In [9]:
#что-то поменялось?
ticket_types = df['ticket_type_nm'].unique()
pd.DataFrame([ [x, df[df.ticket_type_nm==x].shape[0] ] for x in ticket_types], columns=['ticket_type_nm', 'count']).sort_values('count', ascending=False)

,ticket_type_nm,count
3,ТК,489289
0,КОШЕЛЕК,301626
5,СК,145089
4,СК студента,102781
1,СК учащегося,45958
2,СК сотрудника,6278


In [10]:
ticket_enc = OneHotEncoder()
ticket_types_transformed = ticket_enc.fit_transform(df[['ticket_type_nm']]) 
df[ticket_enc.categories_[0]] = ticket_types_transformed.toarray()
df.drop(columns='ticket_type_nm', inplace=True)

station_enc = OneHotEncoder()
station_types_transformed = station_enc.fit_transform(df[['station_id']])
df.drop(columns=['station_id'], inplace=True)
df[station_enc.categories_[0]] = station_types_transformed.toarray()

lines_enc = OneHotEncoder()
lines_id_transformed = lines_enc.fit_transform(df[['line_id']])
df.drop(columns=['line_id'], inplace=True)
df[lines_enc.categories_[0]] = lines_id_transformed.toarray()

/tmp/ipykernel_57045/351205149.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[station_enc.categories_[0]] = station_types_transformed.toarray()
/tmp/ipykernel_57045/351205149.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[station_enc.categories_[0]] = station_types_transformed.toarray()
/tmp/ipykernel_57045/351205149.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [11]:
train, test = train_test_split(df, test_size=0.8, random_state=30)

In [12]:
df.head()

,time_to_under,label,weekday,pass_time,КОШЕЛЕК,СК,СК сотрудника,СК студента,СК учащегося,ТК,...,7,8,9,10,11,12,13,14,15,81
0,216.316667,8001,0,0.25,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,648.183333,9011,0,0.25,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,865.333333,7022,0,0.25,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1048.233333,2022,0,0.25,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,965.600000,2017,0,0.25,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

features = ['pass_time', 'weekday'] + list(ticket_enc.categories_[0]) + list(lines_enc.categories_[0])  + list(station_enc.categories_[0])

X_train = train[features]
y_train = train[['time_to_under']].values.ravel()

param_grid_knn = {
    'alpha': [0.1, 1.0, 5.0]
}

clf = GridSearchCV(estimator = linear_model.Lasso(), param_grid=param_grid_knn, verbose=3, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


[CV 2/2] END .........................alpha=5.0;, score=0.403 total time=   3.2s


/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


[CV 1/2] END .........................alpha=5.0;, score=0.399 total time=   3.7s
[CV 1/2] END .........................alpha=1.0;, score=0.425 total time=   4.1s


...

[CV 2/2] END .........................alpha=1.0;, score=0.430 total time=   4.5s


/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


[CV 2/2] END .........................alpha=0.1;, score=0.433 total time=  12.9s
[CV 1/2] END .........................alpha=0.1;, score=0.428 total time=  12.9s


/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


GridSearchCV(cv=2, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.1, 1.0, 5.0]}, verbose=3)

In [14]:
predict = clf.predict(test[features])

/home/denis/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [15]:
r2_score(test['time_to_under'], predict)

0.43082249597570044